# TinySMPC Tutorial

This tutorial runs through the basics of secure multi-party computation, using TinySMPC.

[Secure multi-party computation](https://en.wikipedia.org/wiki/Secure_multi-party_computation) (SMPC) is a cryptographic technique that allows multiple people to compute a function, where the function itself and its outputs are public, but the inputs are kept private to each person.

A simple example: a group of friends want to compute the total money owned by the group, but without revealing any individual's bank balance.

## Setup

Let's import the only three classes we'll need from TinySMPC.

TinySMPC is written in vanilla Python for understandability, so we don't need any external modules!

In [2]:
!git clone https://github.com/kennysong/tinysmpc.git

Cloning into 'tinysmpc'...
remote: Enumerating objects: 237, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 237 (delta 34), reused 30 (delta 30), pack-reused 197 (from 1)
Receiving objects: 100% (237/237), 68.20 KiB | 1.07 MiB/s, done.
Resolving deltas: 100% (135/135), done.


In [3]:
import sys
sys.path.append('/content/tinysmpc')  # adjust path if changed
from tinysmpc import VirtualMachine, PrivateScalar, SharedScalar



## Creating Multiple Machines

First, let's create a few `VirtualMachines`.

A `VirtualMachine` represents the private computer of a user, and contains data that should not be revealed to other users.

In [4]:
alice = VirtualMachine('alice')
bob = VirtualMachine('bob')
charlie = VirtualMachine('charlie')

Think of Alice, Bob, and Charlie as completely separate computers that can communicate over some network.

(For simplicity, `VirtualMachines` are simulated and all run locally in this notebook kernel.)

Next, let's create private data on each computer.

## Generating Private Data

Private data on a computer is represented in a `PrivateScalar` object. Think of these numbers as each person's bank balance.

In [5]:
a = PrivateScalar(40000, alice)
b = PrivateScalar(15000, bob)
c = PrivateScalar(-20000, charlie)  # Charlie is heavily in debt

Alice can only see her own balance, and cannot see Bob and Charlie's balances.

We can confirm this by looking at the contents of Alice's computer. (Just print `alice`!)

In [6]:
alice

VirtualMachine('alice')
 - PrivateScalar(40000, 'alice')

As we can see, Alice's computer only contains her own `PrivateScalar` that represents her salary.

This is the same for Bob and Charlie's computers.

In [7]:
bob

VirtualMachine('bob')
 - PrivateScalar(15000, 'bob')

In [8]:
charlie

VirtualMachine('charlie')
 - PrivateScalar(-20000, 'charlie')

Next, let's see how we can compute a public function over these private values.

## Secret Sharing

Let's say Alice, Bob, and Charlie want to compute the sum of their bank balances, without revealing their individual balances to anyone.

This is exactly what secure multi-party computation (SMPC) is made for. A technique called "secret sharing" is the fundamental building block of [most](https://crypto.stackexchange.com/questions/80764/is-there-a-secure-multi-party-computation-smpc-scheme-that-doesnt-use-secret) SMPC schemes, so let's talk about that first.

To perform secret sharing, we just call `.share()` on a `PrivateScalar`.

In [9]:
shared_a = a.share([alice, bob, charlie])
shared_a

SharedScalar
 - Share(8098343170359030552, 'alice', Q=None)
 - Share(2030544357309511558, 'bob', Q=None)
 - Share(8317856546041049506, 'charlie', Q=None)

What just happened?

First, TinySMPC used a technique called additive secret sharing to generate three encrypted "shares" of `a`. As you can see above, each share looks like a random number, and it's indeed impossible to find out the true value of `a` from any share (or any two shares).

Then, we sent one encrypted share to each computer. If we look at Bob's machine, he now owns a new `Share` object (that Alice sent him).

In [10]:
bob

VirtualMachine('bob')
 - PrivateScalar(15000, 'bob')
 - Share(2030544357309511558, 'bob', Q=None)

All of the shares that correspond to the original value `a` are tracked in a `SharedScalar` object, which is `shared_a` here.

Reconstructing the value `a` requires bringing all three shares from Alice, Bob, and Charlie.

We can do this easily by calling `.reconstruct()` on a `SharedScalar`.

In [11]:
shared_a.reconstruct(alice)

PrivateScalar(40000, 'alice')

Behind the scenes, TinySMPC sent all shares to Alice, and then reconstructed the original value of `a`. It's the opposite of `.share()`.

Now we know how to use additive secret sharing. But this isn't very useful by itself, so let's look at how to do computation with the shares next.

(If you're curious about how we generate and reconstruct shares, check out [secret_sharing.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/secret_sharing.py).)

## Basic Computation

A magical property of the additive secret sharing scheme is that we can do arithmetic directly on the encrypted shares!

This enables the "computation" in secure multi-party computation.

Let's try doing addition on an encrypted `SharedScalar`.

In [12]:
shared_a = a.share([alice, bob, charlie])
shared_a = shared_a + 5000
shared_a.reconstruct(alice)

PrivateScalar(45000, 'alice')

It worked! Note that we added `5000` directly to the encrypted `SharedScalar`, and it decrypts to the same number as if we added `5000` directly to the original value.

To do this, we applied a special arithmetic protocol directly on each share. For fun, let's inspect the share values directly.

In [13]:
shared_a = a.share([alice, bob, charlie])
print('Shares before adding 5000:')
print(shared_a)

shared_a = shared_a + 5000
print('\nShares after adding 5000:')
print(shared_a)

Shares before adding 5000:
SharedScalar
 - Share(1676128581416852147, 'alice', Q=None)
 - Share(4717499962230162627, 'bob', Q=None)
 - Share(-6393628543646974774, 'charlie', Q=None)

Shares after adding 5000:
SharedScalar
 - Share(1676128581416857147, 'alice', Q=None)
 - Share(4717499962230162627, 'bob', Q=None)
 - Share(-6393628543646974774, 'charlie', Q=None)


If you stare long enough, you'll see that we added `5000` directly to the first share's encrypted value. This is pretty simple (it's not always this easy)

On top of doing addition between a `SharedScalar` and a (public) integer, we can also do addition between `SharedScalars`.

This is how we can securely compute the sum of everyone's bank balances.

In [14]:
shared_a = a.share([alice, bob, charlie])
shared_b = b.share([alice, bob, charlie])
shared_c = c.share([alice, bob, charlie])

shared_sum = shared_a + shared_b + shared_c
shared_sum.reconstruct(bob)

PrivateScalar(35000, 'bob')

Here, we computed the total sum of money owned by everyone, and sent the resulting sum to Bob (who can then share the sum "publicly").

The addition happened directly between encrypted `SharedScalars`, without anyone seeing anyone else's secret values!

Next, let's see how to do more advanced computations (e.g. multiplication, comparison, floating point numbers).

## Advanced Computation

Addition is fun, but we can compute much more complex functions with SMPC as well.

### Multiplication

Let's try out multiplication:

In [15]:
shared_a = PrivateScalar(5, alice).share([alice, bob])
shared_b = PrivateScalar(-10, bob).share([alice, bob])

shared_prod = 2 * shared_b * shared_a
shared_prod.reconstruct(alice)

PrivateScalar(-100, 'alice')

### Exponentiation

Exponentiation works as well (with a public integer in the exponent):

In [16]:
shared_exp1 = shared_prod ** 2
shared_exp2 = shared_prod ** 3

print(shared_exp1.reconstruct(alice))
print(shared_exp2.reconstruct(bob))

PrivateScalar(10000, 'alice')
PrivateScalar(-1000000, 'bob')


### Floating Point Numbers

Last, it's likely that in real-life applications, we'd want to use SMPC on floating point numbers. This is possible!

Recall that our `SharedScalar` and additive secret sharing in general only works on integers, so we need to convert floating point numbers into a fixed-point integer representation first.

Let's see how to do this.

In [17]:
from tinysmpc.fixed_point import fixed_point, float_point

In [18]:
fixed_point(0.005), fixed_point(0.0005), fixed_point(0.00000005), fixed_point(-0.00000005)

(500000, 50000, 5, -5)

As you can likely see, TinySMPC uses an incredibly simple fixed-point representation. We map floats to integers by multiplying by `10^8`, which preserves `8` decimal places of precision.

To get the floating point number back, just use `float_point()`.

In [19]:
float_point(500000), float_point(50000), float_point(5), float_point(-5)

(0.005, 0.0005, 5e-08, -5e-08)

Now, let's see this in action:

In [20]:
a = fixed_point(1.2345)
b = fixed_point(-5.4321)

shared_a = PrivateScalar(a, alice).share([alice, bob])
shared_b = PrivateScalar(b, bob).share([alice, bob])

true_res = (-5.4321 - 1.2345) ** 2
shared_res = (shared_b - shared_a) ** 2
res = shared_res.reconstruct(alice)
res = float_point(res.value, n_mults=1)

res, true_res

(44.44355556, 44.44355556)

It works!

A big disclaimer: fixed-point encoding is finnicky to work with, as you can tell from the unexplained `n_mults` parameter. You'll likely see incorrect results if you don't carefully account for conversion factors and overflows.

Any good, production SMPC library will **transparently** convert and handle floats for you. This adds a lot of codebase complexity, though, and to keep the source code of TinySMPC simple and readable, I decided against implementing this.

However, now you understand the core concept of how SMPC can work on floats!

### Arbitrary Functions

With these tools, we can compute more complex functions as well. For example, we could compute the `exp()` function using its Taylor series.

I'll leave this as an exercise for the reader.

## Limitations

TinySMPC is not intended to be a production SMPC library, so it leaves out several important features.

 - Division of `SharedScalars`. This is possible, but complex to implement (see Algorithm 8 of [the SecureNN paper](https://eprint.iacr.org/2018/442.pdf)).
 - Comparison of `SharedScalars`. Also possible, but complex to implement.
 - `SharedScalars` do not transparently handle floating point numbers.
 - Lack of vectorization, lack of a math library, etc.

We should also talk about the limitations of SMPC in general as a protocol.

- It assumes all participants will honestly apply the right local computations – it's trivial to alter the value of the outcome if you wanted to. This is sometimes called the "[honest-but-curious](https://crypto.stanford.edu/pbc/notes/crypto/sfe.html)" assumption. There are [extensions to SMPC](https://mortendahl.github.io/2017/08/13/secret-sharing-part3/) that provide integrity.
- It requires all participants to stay online. If one machine fails, we cannot continue to compute or decrypt. There are also extensions that solve this problem (search for "k-out-of-n secret sharing").
- It's as slow as your network! A single multiplication requires several round-trip messages between all computers. We'd ideally like the machines to be colocated on a fast local network, but many real-world use cases of SMPC have machines separated over the public internet. SMPC algorithms are often evaluated based on the amount of communication required.

## Recap

To recap, SMPC allows multiple people to compute a function, where the function itself and its outputs are public, but the inputs are kept private to each person.

TinySMPC implements the core building blocks of generating encrypted secret shares, and doing computations directly on that encrypted data.

Here's a summary of the encrypted operations that TinySMPC provides.

|                    | Supported?              | Implementation                                                                          |
|--------------------|-------------------------|-----------------------------------------------------------------------------------------|
| **Addition**       | ✅                       | [SPDZ](https://eprint.iacr.org/2011/535.pdf) algorithm. <br/> See [shared_addition.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_addition.py)             |
| **Subtraction**    | ✅                       | In terms of addition and multiplication.                                                 |
| **Multiplication** | ✅                       | [SPDZ](https://eprint.iacr.org/2011/535.pdf) algorithm.  <br/> See [shared_multiplication.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_multiplication.py) |
| **Division**       | ❌ (too complicated)     | Possible with [SecureNN](https://eprint.iacr.org/2018/442.pdf).                                                                                       |
| **Exponentiation**       | ✅ (public integer only)     | In terms of multiplication.                                                                                       |
| **Greater Than**   | ✅ (public integer only) | [SecureNN](https://eprint.iacr.org/2018/442.pdf) algorithm. <br/> See [shared_comparison.py](https://github.com/kennysong/tinysmpc/blob/master/tinysmpc/shared_comparison.py)     |

What's next?

This was a high-level tutorial. If you want to understand how these protocols work behind the scenes, I encourage you to check out the [source code of TinySMPC](https://github.com/kennysong/tinysmpc)! It was written to be as small and understandable as possible, using vanilla Python and no extra modules.

If you want to use SMPC in production, especially for machine learning, I'd recommend looking at [PySyft](https://github.com/OpenMined/PySyft/) and [TF Encrypted](https://github.com/tf-encrypted/tf-encrypted). They implement useful features such as: vectorization / tensorization, high-level math and ML APIs, transparent float conversion, and more.